# Team Assignment - the descriptive Task

#### First Step: Importing

In [1]:
# ----------- GPU acceleration - use pandas if you don't got a nvidia gpu -----------
import pandas as pd
#import cudf as pd

# install packages with "conda install -c nvidia -c rapidsai -c numba -c conda-forge -c defaults cudf" 
# only on Linux available
# ----------- GPU acceleration - end -----------
import swifter
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("white")
sns.set_palette("GnBu_d")

import folium
from folium import plugins
from folium.plugins import HeatMap
from geopandas.tools import geocode

from datetime import datetime, timedelta

In [2]:
chg17 = pd.read_csv("chicago_2017.csv")

#### Second Step: Validate and review the Data

In [3]:
chg17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3829014 entries, 0 to 3829013
Data columns (total 8 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   start_time          object
 1   end_time            object
 2   start_station_id    int64 
 3   end_station_id      int64 
 4   start_station_name  object
 5   end_station_name    object
 6   bike_id             int64 
 7   user_type           object
dtypes: int64(3), object(5)
memory usage: 233.7+ MB


In [4]:
# chg17.describe()

In [5]:
chg17.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
0,3/31/2017 23:59:07,4/1/2017 00:13:24,66,171,Clinton St & Lake St,May St & Cullerton St,5292,Subscriber
1,3/31/2017 23:56:25,4/1/2017 00:00:21,199,26,Wabash Ave & Grand Ave,McClurg Ct & Illinois St,4408,Subscriber
2,3/31/2017 23:55:33,4/1/2017 00:01:21,520,432,Greenview Ave & Jarvis Ave,Clark St & Lunt Ave,696,Subscriber
3,3/31/2017 23:54:46,3/31/2017 23:59:34,110,142,Dearborn St & Erie St,McClurg Ct & Erie St,4915,Subscriber
4,3/31/2017 23:53:33,4/1/2017 00:00:28,327,331,Sheffield Ave & Webster Ave,Halsted St & Blackhawk St (*),4247,Subscriber


Let's have a look at empty values

In [6]:
emptyRows = chg17[chg17.isna().any(axis=1)]
emptyRows

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type


In [7]:
countEmptyRows = len(chg17[chg17.isna().any(axis=1)])
print("There are {} empty rows".format(countEmptyRows))

There are 0 empty rows


As you can see, there are none empty Values

## Usage Visualization:

First we aggregate our data and do some calculation before

In [ ]:
# Careful, this step takes some minutes
chg17_timeStamps = chg17.copy()
chg17_timeStamps['start_time'] = pd.to_datetime(chg17_timeStamps['start_time'])
chg17_timeStamps['end_time'] = pd.to_datetime(chg17_timeStamps['end_time'])

In [ ]:
chg17_timeStamps.head()

In [ ]:
chg17_timeStamps["date"] = chg17_timeStamps["start_time"].swifter.apply(lambda x: x.date()) 
chg17_timeStamps["travel_time"] = chg17_timeStamps["end_time"] - chg17_timeStamps["start_time"]
chg17_timeStamps["hour"] = chg17_timeStamps["start_time"].swifter.apply(lambda x: x.hour) 
chg17_timeStamps["weekday"] = chg17_timeStamps["start_time"].swifter.apply(lambda x: x.weekday()) 
chg17_timeStamps["week"] = chg17_timeStamps["start_time"].swifter.apply(lambda x: x.isocalendar()[1]) 
chg17_timeStamps["month"] = chg17_timeStamps["start_time"].swifter.apply(lambda x: x.month) 
chg17_timeStamps.sort_values(["date","bike_id"], inplace = True)

In [ ]:
chg17_timeStamps.head()

### Let's take a look at different usages
At first some usage data over the year

In [ ]:
chg17_usage = chg17_timeStamps.groupby(["date"])["bike_id"].count()
chg17_usage = pd.DataFrame(chg17_usage)

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))

ax.plot(chg17_usage, color='darkcyan')

ax.set_title("Usage over a year", fontsize=16)
plt.show()

Breaking this into our to costumer types: costumer, subscriber.

For simplicity reasons we are grouping by week now

In [ ]:
data_range = pd.date_range('2017-01-01', '2017-12-31')

In [ ]:
chg17_usage = chg17_timeStamps.groupby(["week"])["bike_id"].count()
chg17_usage = pd.DataFrame(chg17_usage)

In [ ]:
chg17_usage_subs = chg17_timeStamps[chg17_timeStamps["user_type"]=="Subscriber"].groupby(["week"])["bike_id"].count()
chg17_usage_subs = pd.DataFrame(chg17_usage_subs)

In [ ]:
chg17_usage_costu = chg17_timeStamps[chg17_timeStamps["user_type"]=="Customer"].groupby(["week"])["bike_id"].count()
chg17_usage_costu = pd.DataFrame(chg17_usage_costu)

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))

ax.plot(chg17_usage, label="Total", color='darkcyan')
ax.plot(chg17_usage_subs, label="Subscriber", color='b')
ax.plot(chg17_usage_costu, label="Other Costumer", color='r')

ax.set_title("Usage over a year", fontsize=16)
ax.set_ylabel("Rides in total")
ax.set_xlabel("Weeks")
ax.legend(fontsize=14, loc="upper left")
plt.show()

In [ ]:
chg17_usage = chg17_timeStamps.groupby(["date","month"])["bike_id"].nunique()
chg17_usage = pd.DataFrame(chg17_usage)

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))

sns.boxplot(x = chg17_usage.index.get_level_values(1),
            y = chg17_usage["bike_id"],
            ax=ax)
ax.set_title("Usage over the year", fontsize=16)
plt.show()

Then we take a look at some usage data per individual day. We compare each week day to each other

In [ ]:
chg17_usage = chg17_timeStamps.groupby(["date","weekday"])["bike_id"].nunique()
chg17_usage = pd.DataFrame(chg17_usage)

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))

sns.boxplot(x = chg17_usage.index.get_level_values(1),
            y = chg17_usage["bike_id"],
            ax=ax)
ax.set_title("Usage per Day")
plt.show()

We continue with some analyses of the usage over the day

In [ ]:
chg17_usage = chg17_timeStamps.groupby(["date","hour"])["bike_id"].nunique()
chg17_usage = pd.DataFrame(chg17_usage)

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))

sns.boxplot(x = chg17_usage.index.get_level_values(1),
            y = chg17_usage["bike_id"],
            ax=ax)
ax.set_title("Rental start per Hour")
plt.show()

### We now take a look at availability time
#### Time where the product is not in use is often very important

Here are the station's:

In [3]:
chg17_stations = chg17.copy()

In [4]:
geocode("Märchenstraße 25", provider="nominatim", user_agent='my_request')["geometry"]

0    POINT (7.05897 50.97186)
Name: geometry, dtype: geometry

In [5]:
def getLocation(x):
    try:
        location = geocode(x, provider="nominatim", user_agent='my_request')
        point = location.geometry.iloc[0]
        return f"({round(point.y, 4)}, {round(point.x, 4)})"
    except:
        return "No address found"

In [ ]:
chg17_stations["coordinates"] = chg17_stations["start_station_name"].swifter.apply(lambda x: getLocation(x))

Trying location: Clinton St & Lake St
Trying location: Wabash Ave & Grand Ave
Trying location: Greenview Ave & Jarvis Ave
Trying location: Dearborn St & Erie St
Trying location: Sheffield Ave & Webster Ave
Trying location: Sedgwick St & Webster Ave
Trying location: Daley Center Plaza
Trying location: Desplaines St & Kinzie St
Trying location: Ashland Ave & Division St
Trying location: Wilton Ave & Belmont Ave
Trying location: Wilton Ave & Belmont Ave
Trying location: Southport Ave & Waveland Ave
Trying location: Wabash Ave & Grand Ave
Trying location: Wabash Ave & Roosevelt Rd
Trying location: Sheffield Ave & Fullerton Ave
Trying location: Ashland Ave & Blackhawk St
Trying location: Clinton St & Madison St
Trying location: Millennium Park
Trying location: Clark St & Lake St
Trying location: Clinton St & Washington Blvd
Trying location: Damen Ave & Pierce Ave
Trying location: Clinton St & Lake St
Trying location: McClurg Ct & Illinois St
Trying location: Clinton St & Jackson Blvd
Trying

In [ ]:
chg17_stations.head()

In [ ]:
chg17_stations_map = folium.Map(location=(48.76507, 9.25371),  tiles='Stamen Toner', 
                       zoom_start=12, control_scale=True, max_zoom=20)

for station in chg17_stations:
    folium.CircleMarker(radius=10, location=station, popup='The Waterfront', 
                                 color='crimson', fill_color='crimson').add_to(chg17_stations_map)

In [ ]:
chg17_bikes = chg17["bike_id"].nunique()
chg17_bikes

In [ ]:
chg17_avail = chg17_timeStamps.copy()

In [ ]:
chg17_avail.sort_values(["bike_id","date"], inplace=True)

In [ ]:
chg17_avail.head()

In [ ]:
chg17_avail2 = chg17_avail.groupby(["date","bike_id"])["travel_time"].sum()
chg17_avail2 = pd.DataFrame(chg17_avail2)
chg17_avail2.head(20)